# Neural Network Classification Notebook

## In this notebook, we use neural networks to predict stock percentage values on earnings days


In [1]:
import pandas as pd
import requests
import numpy as np
import matplotlib.pyplot as plt
import re
from datetime import datetime
from pandas.tseries.offsets import BDay
from datetime import datetime, timedelta
import math
import seaborn as sns
import sklearn
import time
import h5py
import copy

from seaborn import set_style
set_style("whitegrid")

import os

from bs4 import BeautifulSoup
import yfinance as yf
from sklearn.model_selection import train_test_split

In [2]:
file_path = 'C:/Users/spatan5/Desktop/New folder/final__master_data.csv'
data = pd.read_csv(file_path)

In [3]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from tensorflow.keras.utils import to_categorical

In [4]:
all_data = data.dropna()

## The 'buckets' for classification are < -2 , -2 to 0, 0 to 2, 2 < (same as logreg)

In [5]:
def bucket(perc_change):
    if perc_change<-2:
        return 0
    elif perc_change<0:
        return 1
    elif perc_change<2:
        return 2
    else:
        return 3

In [6]:
all_data['perc_change_bucket'] = all_data['perc_change_next_prev'].apply(bucket)

C:\Users\spatan5\AppData\Local\Temp\ipykernel_33360\269599157.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['perc_change_bucket'] = all_data['perc_change_next_prev'].apply(bucket)


In [7]:
all_data.drop(columns=['File Name', 'Earnings Date and Time'], inplace=True)

C:\Users\spatan5\AppData\Local\Temp\ipykernel_33360\3457496014.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data.drop(columns=['File Name', 'Earnings Date and Time'], inplace=True)


In [8]:
all_data['month_sin'] = np.sin(2 * np.pi * all_data['Month']/12)
all_data['month_cos'] = np.cos(2 * np.pi * all_data['Month']/12)

C:\Users\spatan5\AppData\Local\Temp\ipykernel_33360\2539933457.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['month_sin'] = np.sin(2 * np.pi * all_data['Month']/12)
C:\Users\spatan5\AppData\Local\Temp\ipykernel_33360\2539933457.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data['month_cos'] = np.cos(2 * np.pi * all_data['Month']/12)


In [9]:
all_data.drop(columns=['Month'], inplace=True)

C:\Users\spatan5\AppData\Local\Temp\ipykernel_33360\3296460.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data.drop(columns=['Month'], inplace=True)


In [10]:
all_data.drop(columns=['perc_change_next_prev'], inplace=True)

C:\Users\spatan5\AppData\Local\Temp\ipykernel_33360\2721240510.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  all_data.drop(columns=['perc_change_next_prev'], inplace=True)


In [11]:
X = all_data.drop('perc_change_bucket', axis=1)
y = all_data['perc_change_bucket']

In [12]:
numeric_features = X.select_dtypes(include=['int64', 'float64']).columns
categorical_features = X.select_dtypes(include=['object']).columns

In [13]:
# Create a column transformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

In [17]:
confusion_matrices = []
predictions = []

for i in range(20):

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
    
    # Create a column transformer with handle_unknown='ignore'
    preprocessor = ColumnTransformer(
        transformers=[
            ('num', StandardScaler(), numeric_features),
            ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
        ])

    
    # Create a preprocessing and training pipeline
    pipeline = Pipeline(steps=[('preprocessor', preprocessor)])
    X_train_prepared = pipeline.fit_transform(X_train)
    X_test_prepared = pipeline.transform(X_test)
    
    y_train = to_categorical(y_train, num_classes=4)
    y_test = to_categorical(y_test, num_classes=4)



    # Model configuration
    input_dim = X_train_prepared.shape[1]  # Number of features
    model = Sequential([
        Dense(128, activation='relu', input_shape=(input_dim,)),
        Dense(32, activation='relu'),
        Dense(4, activation='softmax')  # Output layer for regression
    ])
    
    # Compile the model
    model.compile(optimizer='adam', loss='categorical_crossentropy')

    model.fit(X_train_prepared, y_train, epochs=100, batch_size=32, validation_split=0.2)

    y_pred = model.predict(X_test_prepared)

    # Convert predictions to class labels
    y_pred = np.argmax(y_pred, axis=1)
    
    # Assuming y_test is already categorical and properly encoded,
    # If y_test is one-hot encoded, convert it to class labels as well
    y_test = np.argmax(y_test, axis=1)

    predictions.append([y_pred, y_test])

    cm = confusion_matrix(y_test, y_pred)

    confusion_matrices.append(cm)


    



    

Epoch 1/85


C:\Users\spatan5\Anaconda3\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 1.3770 - val_loss: 1.3625
Epoch 2/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3080 - val_loss: 1.3596
Epoch 3/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2658 - val_loss: 1.3708
Epoch 4/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2391 - val_loss: 1.3759
Epoch 5/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2050 - val_loss: 1.3860
Epoch 6/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1643 - val_loss: 1.3861
Epoch 7/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1190 - val_loss: 1.4047
Epoch 8/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1026 - val_loss: 1.4171
Epoch 9/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0668 - val_loss: 1.4394
Epoch 10/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0377 - val_loss: 1.4527
Epoch 11/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - loss: 0.9994 - val_loss: 1.4659
Epoch 12/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9444 - val_loss: 1.4887


C:\Users\spatan5\Anaconda3\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.3941 - val_loss: 1.3532
Epoch 2/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2886 - val_loss: 1.3532
Epoch 3/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2493 - val_loss: 1.3468
Epoch 4/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2417 - val_loss: 1.3574
Epoch 5/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1934 - val_loss: 1.3550
Epoch 6/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1779 - val_loss: 1.3589
Epoch 7/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1322 - val_loss: 1.3625
Epoch 8/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0870 - val_loss: 1.3770
Epoch 9/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.0599 - val_loss: 1.3921
Epoch 10/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0405 - val_loss: 1.4022
Epoch 11/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0013 - val_loss: 1.4019
Epoch 12/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9550 - val_lo

...

46/46 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 1.3809 - val_loss: 1.3444
Epoch 2/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.3141 - val_loss: 1.3356
Epoch 3/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2736 - val_loss: 1.3309
Epoch 4/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2455 - val_loss: 1.3278
Epoch 5/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.2197 - val_loss: 1.3261
Epoch 6/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1931 - val_loss: 1.3285
Epoch 7/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1632 - val_loss: 1.3195
Epoch 8/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1339 - val_loss: 1.3304
Epoch 9/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1015 - val_loss: 1.3492
Epoch 10/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - loss: 1.0564 - val_loss: 1.3419
Epoch 11/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0373 - val_loss: 1.3588
Epoch 12/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9687 - val_loss: 1.3638


C:\Users\spatan5\Anaconda3\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 9ms/step - loss: 1.3767 - val_loss: 1.3417
Epoch 2/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3217 - val_loss: 1.3330
Epoch 3/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2923 - val_loss: 1.3299
Epoch 4/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2579 - val_loss: 1.3259
Epoch 5/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2449 - val_loss: 1.3258
Epoch 6/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2089 - val_loss: 1.3376
Epoch 7/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1922 - val_loss: 1.3272
Epoch 8/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1180 - val_loss: 1.3368
Epoch 9/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0985 - val_loss: 1.3628
Epoch 10/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0787 - val_loss: 1.3645
Epoch 11/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0324 - val_loss: 1.3812
Epoch 12/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9608 - val_loss: 1.3944


C:\Users\spatan5\Anaconda3\lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 1s 8ms/step - loss: 1.3758 - val_loss: 1.3364
Epoch 2/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.3059 - val_loss: 1.3393
Epoch 3/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2678 - val_loss: 1.3503
Epoch 4/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2442 - val_loss: 1.3550
Epoch 5/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.2036 - val_loss: 1.3544
Epoch 6/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 1.1912 - val_loss: 1.3657
Epoch 7/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1397 - val_loss: 1.3811
Epoch 8/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.1000 - val_loss: 1.3952
Epoch 9/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0640 - val_loss: 1.4019
Epoch 10/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 1.0232 - val_loss: 1.4185
Epoch 11/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 0.9929 - val_loss: 1.4521
Epoch 12/85
46/46 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - loss: 0.9686 - val_lo

## Confusion matrices below, note that (as expected) they are better than regression and like most other models give a measurable approximation of actual % change after earnings

In [18]:
confusion_matrices

[array([[46, 25, 13, 48],
        [20, 16, 15, 27],
        [14, 16, 20, 23],
        [38, 33, 36, 61]], dtype=int64),
 array([[60, 21, 16, 40],
        [17,  9, 23, 30],
        [21, 13, 19, 39],
        [29, 18, 25, 71]], dtype=int64),
 array([[50, 13, 15, 37],
        [19, 23, 20, 28],
        [19, 20, 20, 26],
        [33, 32, 26, 70]], dtype=int64),
 array([[58, 22, 12, 38],
        [15, 18, 14, 28],
        [19, 22, 13, 24],
        [39, 20, 33, 76]], dtype=int64),
 array([[63, 20, 13, 39],
        [28, 14, 14, 28],
        [23, 20, 12, 31],
        [42, 22, 22, 60]], dtype=int64),
 array([[46, 20, 17, 33],
        [21, 20, 19, 22],
        [14, 22, 18, 43],
        [28, 25, 37, 66]], dtype=int64),
 array([[57, 32, 13, 48],
        [21, 18, 17, 33],
        [19, 22, 15, 30],
        [29, 11, 23, 63]], dtype=int64),
 array([[59, 14, 12, 49],
        [21, 15, 15, 34],
        [25,  8, 14, 36],
        [43, 16, 15, 75]], dtype=int64),
 array([[55, 15, 18, 31],
        [25, 15, 18, 2

## Classification reports


In [21]:
for pred in predictions:
    print(classification_report(pred[0], pred[1], labels=[0, 1, 2, 3])

              precision    recall  f1-score   support

           0       0.35      0.39      0.37       118
           1       0.21      0.18      0.19        90
           2       0.27      0.24      0.25        84
           3       0.36      0.38      0.37       159

    accuracy                           0.32       451
   macro avg       0.30      0.30      0.30       451
weighted avg       0.31      0.32      0.31       451

              precision    recall  f1-score   support

           0       0.44      0.47      0.45       127
           1       0.11      0.15      0.13        61
           2       0.21      0.23      0.22        83
           3       0.50      0.39      0.44       180

    accuracy                           0.35       451
   macro avg       0.31      0.31      0.31       451
weighted avg       0.37      0.35      0.36       451

              precision    recall  f1-score   support

           0       0.43      0.41      0.42       121
           1       0.